# Testeo RAG Texto plano con BGE chunkc=1000 y overlap=80%

USAR ENVIRONMENT local_llm_26042926

In [1]:
from langchain import hub
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate

In [2]:
local_llm = "llama3:70b"
url_llm = "http://172.17.30.133:11434" # ordenados con 2 gpus
#url_llm = "http://172.17.30.172:11434" #COMPUTACION

from langchain_community.llms import Ollama
llm = Ollama(model=local_llm, base_url=url_llm, temperature=0) 

In [3]:
# SE USA UNA DE LAS GPU
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {"normalize_embeddings": True}

emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

C:\ProgramData\miniconda3\envs\local_llm_24062024\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\ProgramData\miniconda3\envs\local_llm_24062024\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
from langchain.vectorstores import Chroma
vectorstore_load = Chroma(persist_directory="./chroma_db_1000_bge_rag_texto", embedding_function= emb)

In [5]:
retriever = vectorstore_load.as_retriever(search_type="similarity", search_kwargs={"k": 8})

In [6]:
from langchain import hub
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks and an expert in research projects funded by the European Union under the Horizon 2020 programme.
Use the following context from Horizon 2020 projects to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}

Context: {context} 

Answer:

"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
query_1 = "What is the objective of the project with grant agreement 740934?"
query_2 = "What is the total cost of the project with the acronym HYPERGRYD?"
query_3 = "How much funding was allocated for the project titled Transforming Research through Innovative Practices for Linked interdisciplinary Exploration?"
query_4 = "Which organisation played the role of coordinator in the grant agreement 777998?"
query_5 = "What topic does the project with the acronym INTERRFACE belong to?"
query_6 = "What legal basis was the project titled European Joint Programme on Radioactive Waste Management framed within?"
query_7 = "What type of proposal was the grant agreement 814416?"
query_8 = "To which master call was the project with the acronym G9NIGHT submitted?"
query_9 = "To which sub call was the project titled Electron Nanocrystallography submitted?"
query_10 = "Provide the grant agreement of 1 project which objective is related to artificial intelligence."
query_11 = "Provide the acronym of 1 project which objective is related to robotics."
query_12 = "Provide the title of 1 project which objective is related to geolocation."
query_13 = "Provide the objective of 1 project related to digital twin."
query_14 = "Provide the objective of 3 different projects related to corrosion."
query_15 = "Provide the title of 3 different projects which objective is related to offshore structures."
query_16 = "Provide the acronym of 3 different projects which objective is related to materials engineering."
query_17 = "Provide the grant agreement of 3 different projects which objective is related to nanocomposites."
query_18 = "Provide the name of an organisation that has participated in projects which objective is related to artificial intelligence."
query_19 = "Provide the name of an organisation which activity type is PRC and that has participated in projects which objective is related to robotics."
query_20 = "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
query_21 = "Provide the name of an organisation that has played the role of coordinator in projects which objective is related to digital twin."
query_22 = "Provide the PIC of a Spanish organisation that has participated in projects which objective is related to corrosion."
query_23 = "Provide the name of an european organisation that has participated in projects which objective is related to offshore structures."
query_24 = "Provide the PIC of an european small or medium enterprise that has participated in projects which objective is related to materials engineering."
query_25 = "Provide the name of an european small or medium enterprise that has played the role of coordinator in projects which objective is related to nanocomposites."

### Factuales

In [8]:
retrieved_docs_1 = retriever.get_relevant_documents(query_1)
answer_1 = rag_chain.invoke(query_1) #"What is the objective of the project with grant agreement 740934?"
answer_1

C:\ProgramData\miniconda3\envs\local_llm_24062024\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
C:\ProgramData\miniconda3\envs\local_llm_24062024\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


'The objective of the project with grant agreement 740934, also known as TRIVALENT, is to develop a comprehensive approach to violent radicalization in the EU, combining repressive measures with preventive measures, in collaboration with actors of civil society and communities of reference, while respecting fundamental rights, promoting integration, cultural dialogue, and fighting discrimination.'

ES CORRECTAAA!

In [9]:
retrieved_docs_2 = retriever.get_relevant_documents(query_2)
answer_2 = rag_chain.invoke(query_2) #"What is the total cost of the project with the acronym HYPERGRYD?"
answer_2

"The total cost of the project with the acronym HYPERGRYD is not mentioned in the provided context. I don't know the answer."

NO DA RESPUESTA

In [10]:
retrieved_docs_3 = retriever.get_relevant_documents(query_3)
answer_3 = rag_chain.invoke(query_3) #"How much funding was allocated for the project titled Transforming Research through Innovative Practices for Linked interdisciplinary Exploration?"
answer_3

'The project titled "Transforming Research through Innovative Practices for Linked interdisciplinary Exploration" had a total funded cost of 3498953.75 euros.'

INCORRECTA

In [11]:
retrieved_docs_4 = retriever.get_relevant_documents(query_4)
answer_4 = rag_chain.invoke(query_4) # "Which organisation played the role of coordinator in the grant agreement 777998?"
answer_4

"I don't know the answer as there is no information about the grant agreement 777998 in the provided context."

NO DA RESPUESTA

In [12]:
retrieved_docs_5 = retriever.get_relevant_documents(query_5)
answer_5 = rag_chain.invoke(query_5) # "What topic does the project with the acronym INTERRFACE belong to?"
answer_5

"I don't know the topic of the project with the acronym INTERRFACE as it is not mentioned in the provided context."

NO DA RESPUESTA

In [13]:
retrieved_docs_6 = retriever.get_relevant_documents(query_6)
answer_6 = rag_chain.invoke(query_6) # "What legal basis was the project titled European Joint Programme on Radioactive Waste Management framed within?"
answer_6

'The European Joint Programme on Radioactive Waste Management was framed within the legal basis H2020-EuratomEuratom.'

CORRECTA

In [14]:
retrieved_docs_7 = retriever.get_relevant_documents(query_7)
answer_7 = rag_chain.invoke(query_7) # "What type of proposal was the grant agreement 814416?"
answer_7

"The grant agreement 814416 is not mentioned in the provided context. Therefore, I don't know what type of proposal it was."

NO DA RESPUESTA

In [15]:
retrieved_docs_8 = retriever.get_relevant_documents(query_8)
answer_8 = rag_chain.invoke(query_8) # "To which master call was the project with the acronym G9NIGHT submitted?"
answer_8

"I don't know the master call for the project with the acronym G9NIGHT as it is not mentioned in the provided context."

NO DA RESPUESTA

In [16]:
retrieved_docs_9 = retriever.get_relevant_documents(query_9)
answer_9 = rag_chain.invoke(query_9) # "To which sub call was the project titled Electron Nanocrystallography submitted?"
answer_9

'The project titled Electron Nanocrystallography was submitted to the subcall H2020-MSCA-ITN-2020.'

CORRECTA

### Sobre proyectos y temáticas

In [17]:
retrieved_docs_10 = retriever.get_relevant_documents(query_10)
answer_10 = rag_chain.invoke(query_10) # "Provide the grant agreement of 1 project which objective is related to artificial intelligence."
answer_10

'The grant agreement 876982 corresponds to the project titled AI that composes complex instrumental music for movies, games, advertising and other types of digital media, which acronym is AIVA.'

CORRECTA COMPLETAMENTE

In [18]:
retrieved_docs_11 = retriever.get_relevant_documents(query_11)
answer_11 = rag_chain.invoke(query_11) # "Provide the acronym of 1 project which objective is related to robotics."
answer_11

'The acronym of one project related to robotics is Predict-Plan-Control.'

CORRECTA

In [19]:
retrieved_docs_12 = retriever.get_relevant_documents(query_12)
answer_12 = rag_chain.invoke(query_12) # "Provide the title of 1 project which objective is related to geolocation."
answer_12

'The title of one project related to geolocation is LOCUS (LOCalization and analytics on-demand embedded in the 5G ecosystem, for Ubiquitous vertical applicationS).'

CORRECTA ENTERA

In [20]:
retrieved_docs_13 = retriever.get_relevant_documents(query_13)
answer_13 = rag_chain.invoke(query_13) # "Provide the objective of 1 project related to digital twin."
answer_13

'The objective of the VET-TWIN project related to digital twin is to increase the potential and research capacity of the National Veterinary Research Institute (NVRI) by cooperation with internationally-leading counterparts from Germany and Denmark, specifically in the fields of animal infectious diseases, zoonoses, and food chain safety.'

NO ES CORRECTA, EL ACRONIMO LLEVA TWIN PERO NO DESCRIBE UN DIGITAL TWIN SINO QUE el El objetivo principal mencionado es aumentar la capacidad de investigación del NVRI en colaboración con instituciones de Alemania y Dinamarca, enfocándose en enfermedades infecciosas animales, zoonosis y seguridad alimentaria

In [21]:
retrieved_docs_14 = retriever.get_relevant_documents(query_14)
answer_14 = rag_chain.invoke(query_14) # "Provide the objective of 3 different projects related to corrosion."
answer_14

'Here are the objectives of three different Horizon 2020 projects related to corrosion:\n\n1. EcoGemCoat: The main aim is to develop novel corrosion inhibitors with low toxicity and prepare smart coatings that can protect and detect corrosion.\n\n2. TAMING CORROSION (grant agreement 848794): The objective is to propose a radically new approach to mitigating corrosion of infrastructures, such as bridges and pipelines, by developing cost-effective corrosion mitigation strategies.\n\n3. Unspecified project (grant agreement 101007430): The objective is to develop eco-friendly multifunctional coatings that can protect and detect corrosion based on different mechanisms combining nanostructured inhibiting and sensing additives.'

TODA LA INFORMACIÓN ES CORRECTA

In [22]:
retrieved_docs_15 = retriever.get_relevant_documents(query_15)
answer_15 = rag_chain.invoke(query_15) # "Provide the title of 3 different projects which objective is related to offshore structures."
answer_15

'Here are three different Horizon 2020 projects related to offshore structures:\n\n1. SEAFLOWER - This project proposes strategies to exploit anchor foundations for floating offshore wind turbines.\n2. ASSO - The principal purpose of this project is to establish a sound framework for the feasibility and assessment of adhesive connections for offshore wind installations in the marine environment.\n3. SATH - The aim of this project is to demonstrate in real conditions a floating structure for offshore wind that will allow a reduction in LCOE (Levelized Cost Of Energy) over the current floating technology.'

CORRECTA TOTALMENTE AUNQUE NO DA LOS TITULOS, QUE ES LO QUE SE PIDE

In [23]:
retrieved_docs_16 = retriever.get_relevant_documents(query_16)
answer_16 = rag_chain.invoke(query_16) # "Provide the acronym of 3 different projects which objective is related to materials engineering."
answer_16

'Here are three Horizon 2020 projects related to materials engineering, along with their acronyms:\n\n1. UMA3 (Unique Materials for Advanced Aerospace Applications)\n2. Grade2XL (Application of Functionally Graded Materials to Extra-Large Structures)\n3. MULTI-FUN (Enabling MULTI-FUNctional performance through multi-material additive manufacturing)'

CORRECTA TOTALMENTE

In [24]:
retrieved_docs_17 = retriever.get_relevant_documents(query_17)
answer_17 = rag_chain.invoke(query_17) # "Provide the grant agreement of 3 different projects which objective is related to nanocomposites."
answer_17

'Here are three different projects related to nanocomposites with their grant agreements:\n\n1. Grant agreement 645993 - CO-PILOT project: "Flexible Pilot Scale Manufacturing of Cost-Effective Nanocomposites through Tailored Precision Nanoparticles in Dispersion"\n2. Grant agreement 646397 - NANOLEAP project: "Development of a coordinated network of specialized pilot lines for the production of nanocomposite based products for different civil infrastructure and building applications"\n3. Grant agreement 754890 - NanoComSol project: "Nanocomposite Solutions"'

TODO ES CORRECTO, SALVO EL TITULO DEL SEGUNDO PROYECTO, que en vez del titulo es una descripcion del proyecto

### Preguntas sobre socios

In [25]:
retrieved_docs_18 = retriever.get_relevant_documents(query_18)
answer_18 = rag_chain.invoke(query_18) # "Provide the name of an organisation that has participated in projects which objective is related to artificial intelligence."
answer_18

'The name of an organisation that has participated in projects related to artificial intelligence is FUNDACION INSTITUTO INTERNACIONAL DE INVESTIGACION EN INTELIGENCIA ARTIFICIAL Y CIENCIAS DE LA COMPUTACION.'

COMPROBAR SI ES CORRECTA, ESTA EMPRESA HA PARTICIPADO EN: 871177 y 952176

In [26]:
retrieved_docs_19 = retriever.get_relevant_documents(query_19)
answer_19 = rag_chain.invoke(query_19) # "Provide the name of an organisation which activity type is PRC and that has participated in projects which objective is related to robotics."
answer_19

'The name of an organisation which activity type is PRC and that has participated in projects related to robotics is SMART ROBOTICS BV, ACUTRONIC ROBOTICS SOCIEDAD LIMITADA, ACUTRONIC LINK ROBOTICS AG, PAL ROBOTICS SL.'

TODAS SON PRC, CHECKEAR TEMATICA, AUNQUE PARECE QUE ESTA BIEN

In [27]:
retrieved_docs_20 = retriever.get_relevant_documents(query_20)
answer_20 = rag_chain.invoke(query_20) # "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
answer_20

'The PIC of an organisation that is a small or medium enterprise and has participated in projects related to geolocation is 984514305, which belongs to GEOMOBILE GMBH.'

PIC, NOMBRE Y SME CORRECTO. CHECKEAR TEMATICA AUNQUE PARECE CORRECTO

In [28]:
retrieved_docs_21 = retriever.get_relevant_documents(query_21)
answer_21 = rag_chain.invoke(query_21) # "Provide the name of an organisation that has played the role of coordinator in projects which objective is related to digital twin."
answer_21

"The University of Oxford (THE CHANCELLOR, MASTERS AND SCHOLARS OF THE UNIVERSITY OF OXFORD) and the University of Brighton have played the role of coordinator in projects related to digital twin under Horizon 2020 programme. However, without more specific information about the projects' objectives, it's difficult to pinpoint a single organisation that has worked specifically on digital twin projects."

CHECKEAR

In [29]:
retrieved_docs_22 = retriever.get_relevant_documents(query_22)
answer_22 = rag_chain.invoke(query_22)
answer_22 # "Provide the PIC of a Spanish organisation that has participated in projects which objective is related to corrosion."

'The Spanish organisation that has participated in projects related to corrosion is FUNDACION CENTRO ANDALUZ DE INVESTIGACIONES DEL AGUA, with a PIC of 947977218.'

LA INFORMACION ES CORRECTA, CHEKCERA LA TEMATICA DE SUS PROYECTOS

In [30]:
retrieved_docs_23 = retriever.get_relevant_documents(query_23)
answer_23 = rag_chain.invoke(query_23)
answer_23 # "Provide the name of an european organisation that has participated in projects which objective is related to offshore structures."

"None of the provided context mentions a specific European organisation participating in projects related to offshore structures. However, it's possible that some of the projects mentioned are collaborations between multiple organisations, including European ones. If you're looking for information on specific organisations involved in these projects, I'd be happy to help you search further."

NO DA RESPUESTA

In [31]:
retrieved_docs_24 = retriever.get_relevant_documents(query_24)
answer_24 = rag_chain.invoke(query_24)
answer_24 # "Provide the PIC of an european small or medium enterprise that has participated in projects which objective is related to materials engineering."

'The PIC of a European small or medium enterprise that has participated in projects related to materials engineering is 985560159, which belongs to INTERNATIONAL PROJECT MANAGEMENT PLATING ET MATERIALS SARL based in France. Another example is 983256409, which belongs to ELEMENT MATERIAL TECHNOLOGY SEVILLE SL based in Spain.'

TOTALMENTE CORRECTA

In [32]:
retrieved_docs_25 = retriever.get_relevant_documents(query_25)
answer_25 = rag_chain.invoke(query_25)
answer_25 # "Provide the name of an european small or medium enterprise that has played the role of coordinator in projects which objective is related to nanocomposites."

'The European small or medium enterprise that has played the role of coordinator in projects related to nanocomposites is not explicitly mentioned in the provided context. However, it can be inferred that the coordinators are likely to be research technology organizations (RTOs) or other entities involved in the projects, but their specific names are not provided.'

NO DA RESPUESTA

In [33]:
len(retrieved_docs_25)

8

### Se guardan los resultados en un excel

In [42]:
#!pip install pandas

In [44]:
#!pip install numpy

In [34]:
import pandas as pd
import numpy as np

df = pd.DataFrame()
df['Consulta'] = [query_1, query_2, query_3, query_4, query_5, 
                  query_6, query_7, query_8, query_9, query_10, 
                  query_11, query_12, query_13, query_14, query_15, 
                  query_16, query_17,query_18, query_19, query_20, 
                  query_21, query_22, query_23, query_24, query_25]

df['Retrieved_context_1'] = [retrieved_docs_1[0], retrieved_docs_2[0], retrieved_docs_3[0], retrieved_docs_4[0], retrieved_docs_5[0], 
                             retrieved_docs_6[0], retrieved_docs_7[0], retrieved_docs_8[0], retrieved_docs_9[0], retrieved_docs_10[0], 
                             retrieved_docs_11[0], retrieved_docs_12[0], retrieved_docs_13[0], retrieved_docs_14[0], retrieved_docs_15[0], 
                             retrieved_docs_16[0], retrieved_docs_17[0], retrieved_docs_18[0], retrieved_docs_19[0], retrieved_docs_20[0], 
                             retrieved_docs_21[0], retrieved_docs_22[0], retrieved_docs_23[0], retrieved_docs_24[0], retrieved_docs_25[0]]

df['Retrieved_context_2'] = [
    retrieved_docs_1[1], retrieved_docs_2[1], retrieved_docs_3[1], retrieved_docs_4[1],
    retrieved_docs_5[1], retrieved_docs_6[1], retrieved_docs_7[1], retrieved_docs_8[1],
    retrieved_docs_9[1], retrieved_docs_10[1], retrieved_docs_11[1], retrieved_docs_12[1],
    retrieved_docs_13[1], retrieved_docs_14[1], retrieved_docs_15[1], retrieved_docs_16[1],
    retrieved_docs_17[1], retrieved_docs_18[1], retrieved_docs_19[1], retrieved_docs_20[1],
    retrieved_docs_21[1], retrieved_docs_22[1], retrieved_docs_23[1], retrieved_docs_24[1],
    retrieved_docs_25[1]
]

df['Retrieved_context_3'] = [
    retrieved_docs_1[2], retrieved_docs_2[2], retrieved_docs_3[2], retrieved_docs_4[2],
    retrieved_docs_5[2], retrieved_docs_6[2], retrieved_docs_7[2], retrieved_docs_8[2],
    retrieved_docs_9[2], retrieved_docs_10[2], retrieved_docs_11[2], retrieved_docs_12[2],
    retrieved_docs_13[2], retrieved_docs_14[2], retrieved_docs_15[2], retrieved_docs_16[2],
    retrieved_docs_17[2], retrieved_docs_18[2], retrieved_docs_19[2], retrieved_docs_20[2],
    retrieved_docs_21[2], retrieved_docs_22[2], retrieved_docs_23[2], retrieved_docs_24[2],
    retrieved_docs_25[2]
]

df['Retrieved_context_4'] = [
    retrieved_docs_1[3], retrieved_docs_2[3], retrieved_docs_3[3], retrieved_docs_4[3],
    retrieved_docs_5[3], retrieved_docs_6[3], retrieved_docs_7[3], retrieved_docs_8[3],
    retrieved_docs_9[3], retrieved_docs_10[3], retrieved_docs_11[3], retrieved_docs_12[3],
    retrieved_docs_13[3], retrieved_docs_14[3], retrieved_docs_15[3], retrieved_docs_16[3],
    retrieved_docs_17[3], retrieved_docs_18[3], retrieved_docs_19[3], retrieved_docs_20[3],
    retrieved_docs_21[3], retrieved_docs_22[3], retrieved_docs_23[3], retrieved_docs_24[3],
    retrieved_docs_25[3]
]

df['Retrieved_context_5'] = [
    retrieved_docs_1[4], retrieved_docs_2[4], retrieved_docs_3[4], retrieved_docs_4[4],
    retrieved_docs_5[4], retrieved_docs_6[4], retrieved_docs_7[4], retrieved_docs_8[4],
    retrieved_docs_9[4], retrieved_docs_10[4], retrieved_docs_11[4], retrieved_docs_12[4],
    retrieved_docs_13[4], retrieved_docs_14[4], retrieved_docs_15[4], retrieved_docs_16[4],
    retrieved_docs_17[4], retrieved_docs_18[4], retrieved_docs_19[4], retrieved_docs_20[4],
    retrieved_docs_21[4], retrieved_docs_22[4], retrieved_docs_23[4], retrieved_docs_24[4],
    retrieved_docs_25[4]
]

df['Retrieved_context_6'] = [
    retrieved_docs_1[5], retrieved_docs_2[5], retrieved_docs_3[5], retrieved_docs_4[5],
    retrieved_docs_5[5], retrieved_docs_6[5], retrieved_docs_7[5], retrieved_docs_8[5],
    retrieved_docs_9[5], retrieved_docs_10[5], retrieved_docs_11[5], retrieved_docs_12[5],
    retrieved_docs_13[5], retrieved_docs_14[5], retrieved_docs_15[5], retrieved_docs_16[5],
    retrieved_docs_17[5], retrieved_docs_18[5], retrieved_docs_19[5], retrieved_docs_20[5],
    retrieved_docs_21[5], retrieved_docs_22[5], retrieved_docs_23[5], retrieved_docs_24[5],
    retrieved_docs_25[5]
]

df['Retrieved_context_7'] = [
    retrieved_docs_1[6], retrieved_docs_2[6], retrieved_docs_3[6], retrieved_docs_4[6],
    retrieved_docs_5[6], retrieved_docs_6[6], retrieved_docs_7[6], retrieved_docs_8[6],
    retrieved_docs_9[6], retrieved_docs_10[6], retrieved_docs_11[6], retrieved_docs_12[6],
    retrieved_docs_13[6], retrieved_docs_14[6], retrieved_docs_15[6], retrieved_docs_16[6],
    retrieved_docs_17[6], retrieved_docs_18[6], retrieved_docs_19[6], retrieved_docs_20[6],
    retrieved_docs_21[6], retrieved_docs_22[6], retrieved_docs_23[6], retrieved_docs_24[6],
    retrieved_docs_25[6]
]

df['Retrieved_context_8'] = [
    retrieved_docs_1[7], retrieved_docs_2[7], retrieved_docs_3[7], retrieved_docs_4[7],
    retrieved_docs_5[7], retrieved_docs_6[7], retrieved_docs_7[7], retrieved_docs_8[7],
    retrieved_docs_9[7], retrieved_docs_10[7], retrieved_docs_11[7], retrieved_docs_12[7],
    retrieved_docs_13[7], retrieved_docs_14[7], retrieved_docs_15[7], retrieved_docs_16[7],
    retrieved_docs_17[7], retrieved_docs_18[7], retrieved_docs_19[7], retrieved_docs_20[7],
    retrieved_docs_21[7], retrieved_docs_22[7], retrieved_docs_23[7], retrieved_docs_24[7],
    retrieved_docs_25[7]
]


df['Answer'] = [answer_1, answer_2, answer_3, answer_4, answer_5,
    answer_6, answer_7, answer_8, answer_9, answer_10, 
    answer_11, answer_12, answer_13, answer_14, answer_15,
    answer_16, answer_17, answer_18, answer_19, answer_20, 
    answer_21, answer_22, answer_23, answer_24, answer_25
]


df

,Consulta,Retrieved_context_1,Retrieved_context_2,Retrieved_context_3,Retrieved_context_4,Retrieved_context_5,Retrieved_context_6,Retrieved_context_7,Retrieved_context_8,Answer
0,What is the objective of the project with gran...,page_content='.3.7.Secure societies - Protecti...,page_content='. The grant agreement 740934 was...,page_content='. The grant agreement 740688 was...,page_content='The grant agreement 740934 corre...,page_content='The grant agreement 740362 corre...,page_content='The grant agreement 740723 corre...,page_content='. The grant agreement 740689 was...,page_content='. The grant agreement 740689 was...,The objective of the project with grant agreem...
1,What is the total cost of the project with the...,page_content='. The grant agreement 814881 had...,page_content='The grant agreement 789842 corre...,page_content='. The grant agreement 641544 was...,page_content='. The grant agreement 641495 was...,page_content='. The grant agreement 814881 was...,page_content='The grant agreement 814881 corre...,page_content='. The grant agreement 641495 was...,page_content='The grant agreement 807942 corre...,The total cost of the project with the acronym...
2,How much funding was allocated for the project...,"page_content="". The grant agreement 709443 was...","page_content="". The grant agreement 709443 was...",page_content='. The grant agreement 798866 was...,page_content='.1.3.EXCELLENT SCIENCE - Marie S...,page_content='. The grant agreement 872396 was...,page_content='.1.3.EXCELLENT SCIENCE - Marie S...,page_content='. The grant agreement 795641 had...,page_content='. The grant agreement 795641 was...,"The project titled ""Transforming Research thro..."
3,Which organisation played the role of coordina...,page_content='Information about the organisati...,page_content='Information about the organisati...,page_content='Information about the organisati...,page_content='Information about the organisati...,page_content='Information about the organisati...,page_content='Information about the organisati...,page_content='Information about the organisati...,page_content='Information about the organisati...,I don't know the answer as there is no informa...
4,What topic does the project with the acronym I...,page_content='The grant agreement 654248 corre...,page_content='The grant agreement 730913 corre...,page_content='The grant agreement 714591 corre...,page_content='The grant agreement 664584 corre...,page_content='The grant agreement 730954 corre...,page_content='The grant agreement 665421 corre...,page_content='The grant agreement 731075 corre...,page_content='The grant agreement 101004984 co...,I don't know the topic of the project with the...
5,What legal basis was the project titled Europe...,page_content='. The grant agreement 653951 was...,page_content='The grant agreement 847593 corre...,page_content='The grant agreement 653951 corre...,page_content='. The grant agreement 755554 was...,page_content='. The gran agreement 847593 obje...,page_content='. The Joint Programme will gener...,page_content='. Such Joint Programming will co...,page_content='The grant agreement 945098 corre...,The European Joint Programme on Radioactive Wa...
6,What type of proposal was the grant agreement ...,page_content='The grant agreement 816516 corre...,page_content='The grant agreement 811764 corre...,page_content='. The grant agreement 816516 was...,page_content='The grant agreement 811765 corre...,"page_content='., which acronym is V-LAP. The g...",page_content='The grant agreement 811744 corre...,page_content='The grant agreement 814425 corre...,page_content='. The grant agreement 811764 was...,The grant agreement 814416 is not mentioned in...
7,To which master call was the project with the ...,page_content='The grant agreement 699119 corre...,page_content='. The grant agreement 699119 was...,page_content='. The grant agreement 813999 was...,page_content='. The grant agreement 851809 was...,page_content='. The grant agreement 816149 was

In [47]:
!pip install openpyxl

  Using cached openpyxl-3.1.4-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.4-py2.py3-none-any.whl (251 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [35]:
df.to_excel('Testeo_RAG_Texto_1000_bge_18072024.xlsx', index=False)